In [1]:
import pandas as pd 
pd.set_option("display.max_columns",30)
pd.set_option("display.max_rows",3000)
import matplotlib.pyplot as plt 
import cufflinks as cf 
cf.go_offline()
import numpy as np

### imputacion variables continuas
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp

df = pd.read_csv("titanic.csv")

df.shape

df.survived.value_counts(True)

df

df.survived.value_counts(True)

def completitud_datos_nulos(df):
    return df.isnull().sum().sort_values(ascending=False) / df.shape[0]

completitud_datos_nulos(df)

df = df[(~df["survived"].isnull()) & ~df["pclass"].isnull() & ~df["fare"].isnull() ]

df

def complete_continuous_variables(df,col,strategy='median'):
    X = df[col].copy()
    im = SimpleImputer(strategy=strategy)
    Xi = pd.DataFrame(im.fit_transform(X),columns=col)
    l_ks = []
    for v in col:
        l_ks.append([v,ks_2samp(X[v].dropna(),Xi[v]).statistic])
    ks = pd.DataFrame(l_ks,columns=['feat','ks'])
    #print(ks)
    print((ks.ks>=0.1).sum())
    df[col] = im.transform(df[col].copy())
    return df

complete_continuous_variables(df,["body"])

completitud_datos_nulos(df)

varc = list(df.describe())

varc = [x for x in varc if x not in 'survived']

vard = [x for x in df.columns if x not in varc+['survived']]

tgt = 'survived'

df[tgt].value_counts()

1


survived
0.0    808
1.0    500
Name: count, dtype: int64

In [2]:
# Identificar columnas numéricas y categóricas
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

# Imputar valores nulos en las columnas numéricas (mediana)
num_imputer = SimpleImputer(strategy='median')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

# Imputar valores nulos en las columnas categóricas (moda)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# Verificar que no haya valores nulos después de la imputación
print(df.isnull().sum())


pclass       0
survived     0
name         0
sex          0
age          0
sibsp        0
parch        0
ticket       0
fare         0
cabin        0
embarked     0
boat         0
body         0
home.dest    0
dtype: int64


MODELO CLASIFICACION

In [229]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV


In [230]:
X = df[varc].copy() #TAD "Tabla Analitica de Datos"

In [231]:
y = df[tgt].copy() #variable objetivo

In [232]:
sc = MinMaxScaler()
Xs = pd.DataFrame(sc.fit_transform(X), columns=varc)

In [233]:
y = y.astype(int)

In [235]:
Xt, Xv, yt, yv = train_test_split(Xs,y, train_size=0.7)

In [236]:
def entrenar(param, modelo, X,y):
    grid = RandomizedSearchCV(param_distributions=param,
                             n_jobs=-1,
                             n_iter=10,
                             cv=4,
                             estimator=modelo,
                             error_score='raise')
    grid.fit(X,y)
    return grid, grid.best_estimator_, grid.best_score_, grid.best_params_

In [237]:
def metricas(Xt, Xv, yt, yv, modelo):
    d = {'train':round(roc_auc_score(y_true=yt, y_score=modelo.predict_proba(Xt)[:,1]),3),
         'validate':round(roc_auc_score(y_true=yv, y_score=modelo.predict_proba(Xv)[:,1]),3)
        }
    return d

REDES NEURONALES

In [93]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xt = scaler.fit_transform(Xt)
Xv = scaler.transform(Xv)


In [94]:
param_mlpc = dict(
    hidden_layer_sizes = [(50,), (100,), (150,), (50, 50), (100, 100), (150, 150), (100, 50)],
    activation = ['relu', 'tanh'],
    solver = ['adam'],
    alpha = np.logspace(-5, 0, 20),
    learning_rate = ['adaptive'],
    learning_rate_init = np.logspace(-4, -1, 10)
)

In [95]:
modelo_mlpc = MLPClassifier(max_iter=2000)  # Mayor número de iteraciones para asegurar convergencia

# RandomizedSearchCV para un tuning más eficiente
search = RandomizedSearchCV(modelo_mlpc, param_mlpc, n_iter=50, scoring='accuracy', cv=5, random_state=0, n_jobs=-1)
search.fit(Xt, yt)

best_estimator_mlpc = search.best_estimator_
score_mlpc = search.best_score_
params_mlpc = search.best_params_

# Evaluación del modelo en los datos de validación
metricas(Xt, Xv, yt, yv, best_estimator_mlpc)

{'train': np.float64(0.778), 'validate': np.float64(0.785)}

In [174]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xt = scaler.fit_transform(Xt)
Xv = scaler.transform(Xv)


In [160]:
param_mlpc = dict(hidden_layer_sizes = [(a,b,c,) for a in range(len(varc), len(varc)*2) for b in range(len(varc), len(varc)*2) for c in range(len(varc), len(varc)*2)],
             activation = ['identity', 'logistic', 'tanh', 'relu'],
             solver = ['lbfgs', 'sgd', 'adam'],
             alpha = np.arange(0.0001, 0.001, 0.0001),
             learning_rate = ['constant', 'invscaling', 'adaptive'])

In [162]:
modelo_mlpc = MLPClassifier()
modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo_mlpc)

/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/ve

{'train': np.float64(0.785), 'validate': np.float64(0.789)}

In [175]:
param_mlpc = dict(hidden_layer_sizes = [(a,b,c,) for a in range(len(varc), len(varc)*2) for b in range(len(varc), len(varc)*2) for c in range(len(varc), len(varc)*2)],
             activation = ['identity', 'logistic', 'tanh', 'relu'],
             solver = ['lbfgs', 'sgd', 'adam'],
             alpha = np.arange(0.0001, 0.001, 0.0001),
             learning_rate = ['constant', 'invscaling', 'adaptive'])

In [177]:
modelo_mlpc = MLPClassifier(
    max_iter=6000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo_mlpc)

{'train': np.float64(0.837), 'validate': np.float64(0.726)}

trabajar despues del sobre ajuste

In [566]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xt = scaler.fit_transform(Xt)
Xv = scaler.transform(Xv)


In [567]:
param_mlpc = dict(hidden_layer_sizes = [(a,) for a in range(len(varc), len(varc)+5)],
             activation = ['identity', 'logistic', 'tanh', 'relu'],
             solver = ['lbfgs', 'sgd', 'adam'],
             alpha = np.logspace(-4, 0, 10),
             learning_rate = ['constant', 'invscaling', 'adaptive'])


In [206]:
modelo_mlpc = MLPClassifier(
    max_iter = 5000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo_mlpc)         


{'train': np.float64(0.807), 'validate': np.float64(0.802)}

In [290]:
filename = './modelo_red_neuronal.pkl'
with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc,file)
    
print(f'Modelo guardado en {filename}')

Modelo guardado en ./modelo_red_neuronal.pkl


NUEVO MODELO

In [574]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
Xt = scaler.fit_transform(Xt)
Xv = scaler.transform(Xv)


In [575]:
param_mlpc = dict(hidden_layer_sizes = [(a,) for a in range(len(varc), len(varc)+5)],
             activation = ['identity', 'logistic', 'tanh', 'relu'],
             solver = ['lbfgs', 'sgd', 'adam'],
             alpha = np.logspace(-4, 0, 10),
             learning_rate = ['constant', 'invscaling', 'adaptive'])


In [580]:
modelo_mlpc = MLPClassifier(
    max_iter = 5000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo_mlpc)         


{'train': np.float64(0.8), 'validate': np.float64(0.827)}

In [581]:
filename = './modelo_red_neuronal2.pkl'
with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc,file)
    
print(f'Modelo guardado en {filename}')

Modelo guardado en ./modelo_red_neuronal2.pkl


In [644]:
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import numpy as np

# Escalar los datos
scaler = StandardScaler()
Xt_scaled = scaler.fit_transform(Xt)
Xv_scaled = scaler.transform(Xv)


In [676]:
param_mlpc = dict(hidden_layer_sizes = [(a,b,c,) for a in range(len(varc), len(varc)*2) for b in range(len(varc), len(varc)*2) for c in range(len(varc), len(varc)*2)],
             activation = ['identity', 'logistic', 'tanh', 'relu'],
             solver = ['lbfgs', 'sgd', 'adam'],
             alpha = np.arange(0.00001, 0.001, 0.00001),
             learning_rate = ['constant', 'invscaling', 'adaptive'])

In [677]:
modelo_mlpc = MLPClassifier(
    max_iter = 10000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1
)

modelo_mlpc, best_estimator_mlpc, score_mlpc, params_mlpc = entrenar(param_mlpc, modelo_mlpc, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo_mlpc)         


/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/ve

{'train': np.float64(0.826), 'validate': np.float64(0.808)}

In [365]:
filename = './modelo_red_neuronalfinal.pkl'
with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc,file)
    
print(f'Modelo guardado en {filename}')

Modelo guardado en ./modelo_red_neuronalfinal.pkl


NUEVO MODELO

In [425]:
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

# Balancear las clases con SMOTE
print("Antes de SMOTE:", yt.value_counts())
smote = SMOTE(random_state=42)
Xt_res, yt_res = smote.fit_resample(Xt, yt)
print("Después de SMOTE:", pd.Series(yt_res).value_counts())

# Crear modelos individuales
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
mlp = MLPClassifier(max_iter=5000, random_state=42)

# Crear modelo ensemblado (Voting Classifier)
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('mlp', mlp)
    ],
    voting='soft'  # Usa probabilidades para combinar las predicciones
)

# Entrenar el modelo ensemblado
ensemble.fit(Xt_res, yt_res)

# Evaluar las métricas en los conjuntos de entrenamiento y validación
metrics = {
    'train': round(roc_auc_score(yt_res, ensemble.predict_proba(Xt_res)[:, 1]), 3),
    'validate': round(roc_auc_score(yv, ensemble.predict_proba(Xv)[:, 1]), 3)
}

print("Métricas finales:", metrics)


Antes de SMOTE: survived
0    651
1    395
Name: count, dtype: int64
Después de SMOTE: survived
1    651
0    651
Name: count, dtype: int64
Métricas finales: {'train': np.float64(0.961), 'validate': np.float64(0.819)}


In [426]:
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para balancear clases
print("Antes de SMOTE:", yt.value_counts())
smote = SMOTE(random_state=42)
Xt_res, yt_res = smote.fit_resample(Xt, yt)
print("Después de SMOTE:", pd.Series(yt_res).value_counts())

# Modelos individuales para el ensamble
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, random_state=42)
mlp = MLPClassifier(max_iter=5000, random_state=42, early_stopping=True, validation_fraction=0.1)

# Ensamble con VotingClassifier
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('mlp', mlp)
    ],
    voting='soft'  # Combinación basada en probabilidades
)

# Entrenar el modelo ensamble con los datos balanceados
ensemble.fit(Xt_res, yt_res)

# Evaluar el desempeño en conjunto de entrenamiento y validación
train_auc = roc_auc_score(yt_res, ensemble.predict_proba(Xt_res)[:, 1])
validate_auc = roc_auc_score(yv, ensemble.predict_proba(Xv)[:, 1])

train_f1 = f1_score(yt_res, ensemble.predict(Xt_res))
validate_f1 = f1_score(yv, ensemble.predict(Xv))

# Imprimir métricas
print("Métricas en AUC:")
print("Entrenamiento (train):", round(train_auc, 3))
print("Validación (validate):", round(validate_auc, 3))

print("\nMétricas en F1-Score:")
print("Entrenamiento (train):", round(train_f1, 3))
print("Validación (validate):", round(validate_f1, 3))

# Comprobación de sobreajuste
if (train_auc - validate_auc) > 0.05:
    print("\n⚠️ Alerta: Puede haber indicios de sobreajuste. Ajusta los hiperparámetros.")
else:
    print("\n✅ No se detectó sobreajuste significativo.")


Antes de SMOTE: survived
0    651
1    395
Name: count, dtype: int64
Después de SMOTE: survived
1    651
0    651
Name: count, dtype: int64
Métricas en AUC:
Entrenamiento (train): 0.961
Validación (validate): 0.821

Métricas en F1-Score:
Entrenamiento (train): 0.888
Validación (validate): 0.682

⚠️ Alerta: Puede haber indicios de sobreajuste. Ajusta los hiperparámetros.


In [429]:
mlp = MLPClassifier(
    max_iter=5000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    alpha=0.01
)

rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.05, random_state=42)

ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('mlp', mlp)
    ],
    voting='soft'
)

# Entrenar y evaluar
ensemble.fit(Xt_res, yt_res)

train_auc = roc_auc_score(yt_res, ensemble.predict_proba(Xt_res)[:, 1])
validate_auc = roc_auc_score(yv, ensemble.predict_proba(Xv)[:, 1])

print("Entrenamiento (train):", round(train_auc, 5))
print("Validación (validate):", round(validate_auc, 5))


Entrenamiento (train): 0.83397
Validación (validate): 0.81729


MEJORANDO

In [541]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from scipy.stats import ks_2samp
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import SMOTE

# Configuración inicial
pd.set_option("display.max_columns", 30)
pd.set_option("display.max_rows", 3000)

# Cargar datos
df = pd.read_csv("titanic.csv")

# Función para verificar datos nulos
def completitud_datos_nulos(df):
    return df.isnull().sum().sort_values(ascending=False) / df.shape[0]

# Filtrar datos nulos en columnas clave
df = df[(~df["survived"].isnull()) & ~df["pclass"].isnull() & ~df["fare"].isnull()]

# Imputar valores numéricos y categóricos
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

num_imputer = SimpleImputer(strategy='median')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

# Variables objetivo y predictoras
tgt = 'survived'
varc = list(df.describe())
varc = [x for x in varc if x != tgt]
vard = [x for x in df.columns if x not in varc + [tgt]]

X = df[varc].copy()
y = df[tgt].astype(int).copy()

# Escalado de datos
sc = MinMaxScaler()
Xs = pd.DataFrame(sc.fit_transform(X), columns=varc)

# División de datos
Xt, Xv, yt, yv = train_test_split(Xs, y, train_size=0.8, random_state=42)

# Balanceo de datos con SMOTE
smote = SMOTE(random_state=42)
Xt_res, yt_res = smote.fit_resample(Xt, yt)

# Modelos ajustados
mlp = MLPClassifier(
    max_iter=5000,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    alpha=0.01,
    hidden_layer_sizes=(50, 30, 10)
)

rf = RandomForestClassifier(n_estimators=150, max_depth=6, random_state=42)

gb = GradientBoostingClassifier(
    n_estimators=300,
    learning_rate=0.01,
    max_depth=4,
    random_state=42
)

# Ensamble ajustado
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('gb', gb),
        ('mlp', mlp)
    ],
    voting='soft'
)

# Entrenar el ensamble
ensemble.fit(Xt_res, yt_res)

# Métricas de AUC
train_auc = roc_auc_score(yt_res, ensemble.predict_proba(Xt_res)[:, 1])
validate_auc = roc_auc_score(yv, ensemble.predict_proba(Xv)[:, 1])

print("Entrenamiento (train):", round(train_auc, 3))
print("Validación (validate):", round(validate_auc, 3))

# Validación cruzada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(ensemble, Xt_res, yt_res, scoring='roc_auc', cv=cv)
print("Promedio de AUC en validación cruzada:", round(np.mean(cross_val_scores), 3))


Entrenamiento (train): 0.849
Validación (validate): 0.846
Promedio de AUC en validación cruzada: 0.8


RANDOM FOREST

In [97]:
param = dict(n_estimators=list(range(1, 100, 25)),
                                    criterion=['gini', 'entropy'],
                                    max_depth=[x for x in list(range(2, 5))] + [None],
                                    min_samples_split=[x for x in list(range(2, 4))],
                                    min_samples_leaf=[x for x in list(range(2, 4))],
                                    max_features=[None] + [i * .05 for i in list(range(2, 4))],
                                    max_leaf_nodes=list(range(2, 10)) + [None],
                                    min_impurity_decrease=[x * .10 for x in list(range(2, 4))],
                                    oob_score=[True,False],
                                    warm_start=[True, False],
                                    class_weight=[None, 'balanced'],
                                    max_samples=[None],)

In [98]:
modelo = RandomForestClassifier()
modelo, best_estimator, score, params = entrenar(param, modelo, Xt, yt)
metricas(Xt,Xv,yt,yv,modelo)

/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/home/javier/Documentos/Mineria De Datos/Repositorio/DMUNAM2025/venv/lib/python3.12/site-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the

{'train': np.float64(0.5), 'validate': np.float64(0.5)}

In [ ]:
# Parámetros para Random Forest
param_rf_corrected = {
    'n_estimators': list(range(50, 200, 50)),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'], 
    'class_weight': [None, 'balanced']
}


modelo_rf_corrected = RandomForestClassifier()
modelo_rf_corrected, best_estimator_rf_corrected, score_rf_corrected, params_rf_corrected = entrenar(param_rf_corrected, modelo_rf_corrected, Xt, yt)
metrics_rf_corrected = metricas(Xt, Xv, yt, yv, modelo_rf_corrected)
metrics_rf_corrected

{'train': np.float64(0.926), 'validate': np.float64(0.807)}

QUITANDO SOBRE AJUSTE

In [ ]:
# Parámetros para Random Forest
param_rf_corrected = {
    'n_estimators': list(range(50, 200, 50)),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'], 
    'class_weight': [None, 'balanced']
}


modelo_rf_corrected = RandomForestClassifier()
modelo_rf_corrected, best_estimator_rf_corrected, score_rf_corrected, params_rf_corrected = entrenar(param_rf_corrected, modelo_rf_corrected, Xt, yt)
metrics_rf_corrected = metricas(Xt, Xv, yt, yv, modelo_rf_corrected)
metrics_rf_corrected

{'train': np.float64(0.889), 'validate': np.float64(0.816)}

In [ ]:

param_rf_adjusted = {
    'n_estimators': [50, 100], 
    'criterion': ['gini'],  
    'max_depth': [10, 15],  
    'min_samples_split': [10, 15], 
    'min_samples_leaf': [4, 6], 
    'max_features': ['sqrt'],  
    'class_weight': [None, 'balanced']  
}


modelo_rf_adjusted = RandomForestClassifier()
modelo_rf_adjusted, best_estimator_rf_adjusted, score_rf_adjusted, params_rf_adjusted = entrenar(param_rf_adjusted, modelo_rf_adjusted, Xt, yt)
metrics_rf_adjusted = metricas(Xt, Xv, yt, yv, modelo_rf_adjusted)

metrics_rf_adjusted


{'train': np.float64(0.884), 'validate': np.float64(0.819)}

In [ ]:

param_rf_adjusted = {
    'n_estimators': [50, 100],  
    'criterion': ['gini'], 
    'max_depth': [8, 10],  
    'min_samples_split': [10, 15],  
    'min_samples_leaf': [5, 7],  
    'max_features': ['sqrt'],  
    'class_weight': [None, 'balanced']  
}


modelo_rf_adjusted = RandomForestClassifier()
modelo_rf_adjusted, best_estimator_rf_adjusted, score_rf_adjusted, params_rf_adjusted = entrenar(param_rf_adjusted, modelo_rf_adjusted, Xt, yt)
metrics_rf_adjusted = metricas(Xt, Xv, yt, yv, modelo_rf_adjusted)

metrics_rf_adjusted


{'train': np.float64(0.857), 'validate': np.float64(0.811)}

In [ ]:

param_rf_adjusted = {
    'n_estimators': [60, 80],  
    'criterion': ['gini'],  
    'max_depth': [8, 10],  
    'min_samples_split': [20, 25],  
    'min_samples_leaf': [5, 7],  
    'max_features': ['sqrt'],  
    'class_weight': ['balanced']  
}


modelo_rf_adjusted = RandomForestClassifier()
modelo_rf_adjusted, best_estimator_rf_adjusted, score_rf_adjusted, params_rf_adjusted = entrenar(param_rf_adjusted, modelo_rf_adjusted, Xt, yt)
metrics_rf_adjusted = metricas(Xt, Xv, yt, yv, modelo_rf_adjusted)

metrics_rf_adjusted


{'train': np.float64(0.853), 'validate': np.float64(0.812)}

In [ ]:

param_rf_adjusted = {
    'n_estimators': [40, 100], 
    'criterion': ['gini'],  
    'max_depth': [5, 7],  
    'min_samples_split': [10, 15], 
    'min_samples_leaf': [5, 7],  
    'max_features': ['sqrt'],  
    'class_weight': ['balanced'] 
}


modelo_rf_adjusted = RandomForestClassifier()
modelo_rf_adjusted, best_estimator_rf_adjusted, score_rf_adjusted, params_rf_adjusted = entrenar(param_rf_adjusted, modelo_rf_adjusted, Xt, yt)
metrics_rf_adjusted = metricas(Xt, Xv, yt, yv, modelo_rf_adjusted)

metrics_rf_adjusted


{'train': np.float64(0.822), 'validate': np.float64(0.808)}

In [363]:
import pickle

In [364]:
filename = './randomforest.pkl'
with open(filename, 'wb') as file:
    pickle.dump(modelo_mlpc,file)
    
print(f'Modelo guardado en {filename}')

Modelo guardado en ./randomforest.pkl
